## Hipótesis 4: Sobre el Poder Predictivo del Machine Learning 

- **Hipótesis Nula (H₀)**: Un modelo de Machine Learning, usando los indicadores como características, no es capaz de generar una señal de trading que produzca un rendimiento superior al de la mejor estrategia basada en reglas que encontramos.


- **Hipótesis Alternativa (H₁)**: Un modelo de Machine Learning sí puede aprender patrones complejos para generar una señal más inteligente, logrando un rendimiento superior y/o un mejor rendimiento ajustado al riesgo que cualquier estrategia de reglas fijas.


**Objetivo del Notebook**

Este notebook tiene como objetivo desarrollar un modelo de Machine Learning capaz de generar señales de compra superiores a la mejor estrategia basada en reglas identificada en el análisis anterior (signal_ichimoku_kijun_cross con un retorno del 181.63%). Se probarán y evaluarán múltiples algoritmos de clasificación para encontrar el modelo con el mejor rendimiento predictivo y, finalmente, se guardará el modelo campeón para su posterior evaluación en un backtest.



 **Interpretación de las Métricas de Evaluación de Modelos**

Para evaluar el rendimiento de nuestros modelos de clasificación, utilizamos un **Informe de Clasificación** estándar, que proporciona varias métricas clave.  
Dado que nuestro objetivo es generar señales de compra fiables, nos centramos principalmente en el rendimiento del modelo para la **clase 1** (que representa una oportunidad de compra exitosa).

---

 **Métricas Principales (Clase 1)**

Estas métricas evalúan la capacidad del modelo para identificar correctamente las oportunidades de inversión.

**Precision** (Precisión)
- **Pregunta que responde:**  
  De todas las veces que el modelo predijo "**¡Compra!**", ¿qué porcentaje de veces acertó?

- **Relevancia en el Proyecto:**  
  Es la métrica más importante. Una alta precisión es crucial para una estrategia de trading, ya que minimiza el capital desperdiciado en falsas alarmas y operaciones perdedoras.

**Recall (Sensibilidad)**
- **Pregunta que responde:**  
  De todas las oportunidades de compra reales que hubo, ¿cuántas fue capaz de encontrar el modelo?

- **Relevancia en el Proyecto:**  
  Mide la capacidad del modelo para no perderse ganancias potenciales.  
  Existe un *trade-off* inherente entre la **Precisión** y el **Recall**.

**F1-Score**
- **Pregunta que responde:**  
  ¿Cuál es el equilibrio armónico entre la **Precisión** y el **Recall**?

- **Relevancia en el Proyecto:**  
  Es una métrica global útil para comparar modelos que tienen diferentes equilibrios de Precisión/Recall.

 **Support**
- **Qué significa:**  
  Es simplemente el número total de ejemplos reales de esa clase en el conjunto de datos de prueba.

---

**Métricas Globales**

 **Accuracy (Exactitud)**
- **Qué significa:**  
  De todas las predicciones realizadas, ¿qué porcentaje fue correcto?

- **Nota:**  
  Esta métrica puede ser engañosa en datasets desbalanceados como el nuestro.  
  Un modelo inútil que siempre predice "**No Compra**" podría tener una exactitud alta.

 **Macro Avg & Weighted Avg**
- **Qué significan:**  
  Son las medias de las métricas (**Precisión**, **Recall**, **F1**) a través de ambas clases.  
  La **Media Ponderada (Weighted Avg)** es más relevante para nosotros, ya que ajusta el cálculo según el número de ejemplos en cada clase.


### 1. Configuración del Entorno


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier


### 2. Preparación de Datos para Machine Learning
- El éxito de cualquier modelo de ML depende de la calidad de los datos con los que se entrena. 
- En esta fase, definiremos el problema a resolver (la variable target), seleccionaremos las características más informativas y prepararemos los datos para el entrenamiento.

#### 2.1. Carga y Enriquecimiento de Datos
- Se carga el dataset pre-procesado del notebook anterior, que ya contiene los indicadores técnicos calculados y el ``target``.
- el target que establecimos fue si el precio subiria un 4% en los proximos 19 dias

In [ ]:
df = pd.read_csv('../data/VOO_ind_signal.csv',parse_dates=['date'])

In [ ]:
df.head()

- Comprobamo el balance de las clases del target

In [ ]:
print(df['target'].value_counts(normalize=True))

#### 2.2. Selección de Características (Feature Selection)
Basado en el análisis de correlación del EDA, se selecciona un subconjunto de características potente y poco redundante. Se incluyen las relaciones numéricas de nuestra estrategia campeona (Ichimoku) para darle al modelo la mejor información posible.


In [ ]:
#Definicion de features con diferentes tipos para despues combinarlas. 

features_num = [
    'rsi', 'macd_diff', 'bollinger_width', 'atr', 'adx', 
    'volume_ratio', 'price_vs_kijun', 'tenkan_vs_kijun'
]

features_bool = [
    'signal_ema_price','signal_macd_buy',
    'signal_stochastic_buy','signal_ichimoku_kijun_cross'
]

features_advan = ['body_size','upper_wick','lower_wick','rsi_roc_5',
    'month_sin','month_cos']


- Para el entrenamiendo de diferentes modelos definimos diferentes tipos de features y despues las utilizaremos combianas, con esto lo que queremos averiguar es: 
    1) Cual de ellas tendra mejor rendimiento
    2) ¿ Combinar las features nos da mejor rendimiento? o ¿ proporcionas mas ruido y peores rendimientos?

In [ ]:
# Creamos las X e y para las features_num (solamente)
X_num = df[features_num]
y = df['target'] # el traget al ser el mismo solo lo definimos una vez. 

# Creamos las X e y para la combinacion de features_num + features_bol
features_num_bol = features_num + features_bool
X_num_bol = df[features_num_bol]

# Creamos la combinacion de features_num + features_advan
features_num_advan = features_num + features_advan
X_num_advan = df[features_num_advan]

#### 2.4. División y Escalado de Datos
- Se dividen los datos para cada tipo de features. 
- Despues hace el escalado de los datos y guardado en las diferentes carpetas donde guaradaremos los modelos entrenados. 

In [ ]:
#Division de los datos
X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(X_num, y, test_size=0.2, shuffle=False)

X_train_num_bol, X_test_num_bol, y_train_num_bol,y_test_num_bol = train_test_split(X_num_bol,y,test_size=0.2,shuffle=False)

X_train_num_advan,X_test_num_advan,y_train_num_advan, y_test_num_advan = train_test_split(X_num_advan,y,test_size=0.2,shuffle=False)

In [ ]:
#Escalado para los datos de features_num
scaler_num = StandardScaler()
X_train_scaled_num = scaler_num.fit_transform(X_train_num)
X_test_scaled_num = scaler_num.transform(X_test_num)

with open('../models/models_num/scaler_num.pkl', 'wb') as f:
    pickle.dump(scaler_num, f)


#Escalado para los datos de features_num_bol
scaler_numbol = StandardScaler()
X_train_scaled_numbol = scaler_numbol.fit_transform(X_train_num_bol)
X_test_scaled_numbol = scaler_numbol.transform(X_test_num_bol)


with open('../models/models_numbols/scaler_numbol.pkl', 'wb') as f:
    pickle.dump(scaler_numbol, f)

#Escalado para los datos de features_num_advan
scaler_numadvan = StandardScaler()
X_train_scaled_numadvan = scaler_numadvan.fit_transform(X_train_num_advan)
X_test_scaled_numadvan = scaler_numadvan.transform(X_test_num_advan)


with open('../models/models_numadvan/scaler_numadvan.pkl', 'wb') as f:
    pickle.dump(scaler_numadvan, f)

### 3. Entrenamiento de modelos. (SIN AJUSTE DE HIPERPARAMETROS)

In [ ]:
def train_evaluate_models (model, model_name:str, X_train, y_train, X_test, y_test,save_model:bool=False, save_dir : str ='models'):
    
    print(f"Entrenando modelo de : {model_name}")
    
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    print(f"Informe de Clasisficación:{model_name}")
    print(classification_report(y_test,y_pred))
    
    print(f"Matriz de Confusión de {model_name}")
    ConfusionMatrixDisplay.from_estimator(model,X_test,y_test,cmap='Blues')
    plt.title(f'Matriz de Confusión - {model_name}')
    plt.show()
    
    if save_model: 
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            print(f"Carpeta '{save_dir} creada ")
        filepath = os.path.join(save_dir,f'{model_name}.pkl')
        with open(filepath,'wb') as f: 
            pickle.dump(model,f)
        print(f'El modelo se ha guardado con exito')

    return model 

#### 3.1 Entrenamiento de modelos con ``class_weight='balanced'`` 

#### 3.1.1 Regresion logistica

In [ ]:
log_reg_num = LogisticRegression(random_state=42,class_weight='balanced',max_iter=1000)
log_reg_num_train = train_evaluate_models(
    model=log_reg_num,
    model_name='Regresion_logistica_num',
    X_train=X_train_scaled_num,
    y_train=y_train_num,
    X_test=X_test_scaled_num,
    y_test=y_test_num,
    save_model=True,
    save_dir='../models/models_num/'
)

In [ ]:
log_reg_numbol = LogisticRegression(random_state=42,class_weight='balanced',max_iter=1000)
log_reg_numbol_train = train_evaluate_models(
    model=log_reg_numbol,
    model_name='Regresion_logistica_numbol',
    X_train=X_train_scaled_numbol,
    y_train=y_train_num_bol,
    X_test=X_test_scaled_numbol,
    y_test=y_test_num_bol,
    save_model=True,
    save_dir='../models/models_numbols/'
)

In [ ]:
log_reg_numadvan = LogisticRegression(random_state=42,class_weight='balanced',max_iter=1000)
log_reg_numadvan_train = train_evaluate_models(
    model=log_reg_numadvan,
    model_name='Regresion_logistica_numbol',
    X_train=X_train_scaled_numadvan,
    y_train=y_train_num_advan,
    X_test=X_test_scaled_numadvan,
    y_test=y_test_num_advan,
    save_model=True,
    save_dir='../models/models_numadvan/'
)

#### 3.1.2 Random Forest


In [ ]:
rf_num = RandomForestClassifier(
    n_estimators=100,       
    random_state=42,
    class_weight='balanced', 
    n_jobs=-1               
)
rf_num_train = train_evaluate_models(
    model=rf_num,
    model_name='Random Forest Num',
    X_train=X_train_scaled_num,
    y_train=y_train_num,
    X_test=X_test_scaled_num,
    y_test=y_test_num,
    save_model=True,
    save_dir='../models/models_num/'
)


In [ ]:
rf_numbol = RandomForestClassifier(
    n_estimators=100,       
    random_state=42,
    class_weight='balanced', 
    n_jobs=-1               
)

rf_numbol_train = train_evaluate_models(
    model=rf_numbol,
    model_name='Random_Forest_NumBol',
    X_train=X_train_scaled_numbol,
    y_train=y_train_num_bol,
    X_test=X_test_scaled_numbol,
    y_test=y_test_num_bol,
    save_model=True,
    save_dir='../models/models_numbols/'
)


In [ ]:
rf_numadvan = RandomForestClassifier(
    n_estimators=100,       
    random_state=42,
    class_weight='balanced', 
    n_jobs=-1               
)

rf_numadvan_train = train_evaluate_models(
    model=rf_numadvan,
    model_name='Random_Forest_NumAdv',
    X_train=X_train_scaled_numadvan,
    y_train=y_train_num_advan,
    X_test=X_test_scaled_numadvan,
    y_test=y_test_num_advan,
    save_model=True,
    save_dir='../models/models_numadvan/'
)


#### 3.1.3 Suport Vector Machine (SVM)

In [ ]:
svm_num = SVC(
    probability=True,       
    random_state=42,
    class_weight='balanced'
)

svm_num_train = train_evaluate_models(
    model=svm_num,
    model_name='Support_Vector_Machine_Num',
    X_train=X_train_scaled_num,
    y_train=y_train_num,
    X_test=X_test_scaled_num,
    y_test=y_test_num,
    save_model=True,
    save_dir='../models/models_num/'
)

In [ ]:
svm_numbol = SVC(
    probability=True,       
    random_state=42,
    class_weight='balanced'
)

svm_numbol_train = train_evaluate_models(
    model=svm_numbol,
    model_name='Support_vector_Machine_NumBol',
    X_train=X_train_scaled_numbol,
    y_train=y_train_num_bol,
    X_test=X_test_scaled_numbol,
    y_test=y_test_num_bol,
    save_model=True,
    save_dir='../models/models_numbols/'
)  


In [ ]:
svm_numadvan = SVC(
    probability=True,       
    random_state=42,
    class_weight='balanced'
)

svm_numadvan_train = train_evaluate_models(
    model=svm_numadvan,
    model_name='Support_Vector_machine_numAdvanc',
    X_train=X_train_scaled_numadvan,
    y_train=y_train_num_advan,
    X_test=X_test_scaled_numadvan,
    y_test=y_test_num_advan,
    save_model=True,
    save_dir='../models/models_numadvan/'
)    


#### 3.2 Entrenamiento de modelos con ``SMOTE`` 

In [ ]:
smote_num = SMOTE(random_state=42)
X_train_num_resampled, y_train_num_resampled = smote_num.fit_resample(X_train_scaled_num, y_train_num)

smote_numbool = SMOTE(random_state=42)
X_train_numbool_resampled, y_train_numbool_resampled = smote_numbool.fit_resample(X_train_scaled_numbol, y_train_num_bol)

smote_numadvan = SMOTE(random_state=42)
X_train_numadvan_resampled, y_train_numadvan_resampled = smote_numadvan.fit_resample(X_train_scaled_numadvan,y_train_num_advan)

#### 3.2.1 GRADIEN BOOSTING


In [ ]:
gb_model = GradientBoostingClassifier(
    n_estimators=1000,       
    random_state=42,
    learning_rate=0.05,
    max_depth=3
)

In [ ]:
gb_model_train_num = train_evaluate_models(
    model=gb_model,
    model_name='Gradien_Boosting_num',
    X_train=X_train_num_resampled,     
    y_train=y_train_num_resampled,      
    X_test=X_test_scaled_num,           
    y_test=y_test_num,
    save_model=True,
    save_dir='../models/models_num/'
)

In [ ]:
gb_model_train_numbol = train_evaluate_models(
    model=gb_model,
    model_name='Gradient_boosting_numbol',
    X_train=X_train_numbool_resampled,
    y_train=y_train_numbool_resampled,
    X_test=X_test_scaled_numbol,
    y_test=y_test_num_bol,
    save_model=True,
    save_dir='../models/models_numbols/'
    )

In [ ]:
gb_model_train_numadvam = train_evaluate_models(
    model=gb_model,
    model_name='Gradient_boosting_numadvan',
    X_train=X_train_numadvan_resampled,
    y_train=y_train_numadvan_resampled,
    X_test=X_test_scaled_numadvan,
    y_test=y_test_num_advan,
    save_model=True,
    save_dir='../models/models_numadvan/'
    )

#### 3.2.2 XGBOOST

In [ ]:
xgb_model = XGBClassifier(
    n_estimators=1000,
    random_state=42, 
    eval_metric='logloss',  
    n_jobs=-1
)

In [ ]:
xgb_model_num_train = train_evaluate_models(
    model=xgb_model,
    model_name='XGBoost_num',
    X_train=X_train_num_resampled,     
    y_train=y_train_num_resampled,      
    X_test=X_test_scaled_num,           
    y_test=y_test_num,
    save_model=True,
    save_dir='../models/models_num/'
)


In [ ]:
xgb_model_numbol_train = train_evaluate_models(
    model=xgb_model,
    model_name='XGBoost_numBol',
    X_train=X_train_numbool_resampled,     
    y_train=y_train_numbool_resampled,      
    X_test=X_test_scaled_numbol,           
    y_test=y_test_num_bol,
    save_model=True,
    save_dir='../models/models_numbols/'
)

In [ ]:
xgb_model_numadvan_train = train_evaluate_models(
    model=xgb_model,
    model_name='XGBoost_numadvab',
    X_train=X_train_numadvan_resampled,     
    y_train=y_train_numadvan_resampled,      
    X_test=X_test_scaled_numadvan,           
    y_test=y_test_num_advan,
    save_model=True,
    save_dir='../models/models_numadvan/'
)

#### 3.3 Modelo de DeepLearning: MLP Classifier

In [ ]:
mlp_model = MLPClassifier(
    hidden_layer_sizes=(50, 25), 
    random_state=42,
    max_iter=1000, 
    activation='relu'
)

In [ ]:
mlp__model_num_trained = train_evaluate_models(
    model=mlp_model,
    model_name="Red Neuronal (MLP_num)",
    X_train=X_train_num_resampled,       
    y_train=y_train_num_resampled,       
    X_test=X_test_scaled_num,            
    y_test=y_test_num,
    save_model=True,
    save_dir='../models/models_num/'
)

In [ ]:
mlp__model_numbol_trained = train_evaluate_models(
    model=mlp_model,
    model_name="Red Neuronal (MLP_numbol)",
    X_train=X_train_numbool_resampled,
    y_train=y_train_numbool_resampled,
    X_test=X_test_scaled_numbol,
    y_test=y_test_num_bol,
    save_model=True,
    save_dir='../models/models_numbols/'
)

In [ ]:
mlp__model_numadvan_trained = train_evaluate_models(
    model=mlp_model,
    model_name="Red Neuronal (MLP_numadvan)",
    X_train=X_train_numadvan_resampled,       
    y_train=y_train_numadvan_resampled,       
    X_test=X_test_scaled_numadvan,            
    y_test=y_test_num_advan,
    save_model=True,
    save_dir='../models/models_numadvan/'
)

In [ ]:
def generate_summary_report (model_dir:str,X_test,y_test)->pd.DataFrame:
    """
    Carga todos los modelos .pkl de un directorio, los evalúa y devuelve un 
    DataFrame con un resumen completo de métricas, incluyendo los componentes
    de la matriz de confusión.
    """
    result_list=[]
    
    
    

### 5. Ajuste de Hiper parametros


- Para el ajuste de hiper parametros se selecciona Random Forest y XGboost ya que han sido los que mejor resultado dan. 

#### 5.1 Ajuste de hiperparametros en Random Forest. 

In [ ]:
param_Hiper_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 7, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 0.5],
    'class_weight': ['balanced']
}

In [ ]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
random_search_rf = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_Hiper_rf,
    n_iter=50,  
    cv=5,
    scoring='precision', 
    n_jobs=-1,
    verbose=2,
    random_state=42
)

In [ ]:
random_search_rf.fit(X_train_scaled_adv, y_train)

In [ ]:
y_pred_best_rf = random_search_rf.best_estimator_.predict(X_test_scaled_adv)

In [ ]:
print(random_search_rf.best_params_)

In [ ]:
print("Informe de Clasificación ajustada Random Forest)")
print(classification_report(y_test, y_pred_best_rf))


In [ ]:
ConfusionMatrixDisplay.from_estimator(random_search_rf, X_test_scaled_adv, y_test, cmap='magma')
plt.title('Matriz de Confusión - Modelo Final Random Forest')
plt.show()

XGBOOST

In [ ]:
param_dist_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.05, 0.1, 0.2, 0.3],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

In [ ]:
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1)


random_search_xgb = (
    estimator=xgb,
    param_distributions=param_dist_xgb,
    n_iter=100,
    cv=5,
    scoring=['precision', 'recall', 'f1'],
    n_jobs=-1,
    verbose=2,
    random_state=42
)


random_search_xgb.fit(X_train_resamp_adv, y_train_resamp_adv)


print("\n--- Búsqueda para XGBoost completada ---")
print("Mejores parámetros encontrados:")
print(random_search_xgb.best_params_)

best_xgb_model = random_search_xgb.best_estimator_
y_pred_best_xgb = best_xgb_model.predict(X_test_scaled_adv)

print("\n--- Informe de Clasificación (XGBoost OPTIMIZADO) ---")
print(classification_report(y_test, y_pred_best_xgb))